In [ ]:
from xml.dom import minidom
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
input_path_xml = r'data\interim\trackmate_tests\correct_labels_and_units_v3_tracked.xml'

In [ ]:
mydoc = minidom.parse(input_path_xml)

In [ ]:
feature_names = ['ID', 'FRAME', 'ESTIMATED_DIAMETER',  'POSITION_X', 'POSITION_Y', 'POSITION_Z', 'MEDIAN_INTENSITY']


In [ ]:
data = []
for spot in tqdm(mydoc.getElementsByTagName('Spot')):
    data.append([spot.attributes[key].value for key in feature_names])

In [ ]:
spot_data = pd.DataFrame(data, columns=feature_names)

In [ ]:
spot_data

In [ ]:
data = []
for track in tqdm(mydoc.getElementsByTagName('Track')):
    track_id = track.attributes['TRACK_ID'].value
    for edge in track.getElementsByTagName('Edge'):
        data.append([track_id, edge.attributes['SPOT_SOURCE_ID'].value])
        data.append([track_id, edge.attributes['SPOT_TARGET_ID'].value])
                     
track_data = pd.DataFrame(data, columns=['TRACK_ID', 'SPOT_ID'])

In [ ]:
track_data

In [ ]:
track_data = track_data.drop_duplicates()

In [ ]:
# sanity check
assert(np.all(track_data.groupby('SPOT_ID').count().values == 1))

In [ ]:
spot_data = pd.merge(spot_data, track_data, left_on="ID", right_on='SPOT_ID')

In [ ]:
spot_data

In [ ]:
spot_data = spot_data.apply(pd.to_numeric)

In [ ]:
spot_data

In [ ]:
spot_data.groupby('TRACK_ID')

In [ ]:
spot_data['volume'] = (spot_data.ESTIMATED_DIAMETER / 2)**3 * 4/3 * np.pi

In [ ]:
spot_data

In [ ]:
# Sanity check:
# calculation of diameter:
# np.cbrt(p.area*3/(4* np.pi))*2
# area is in voxels instead of um
# voxel size = 0.400, 0.063, 0.063


voxel_volume_um = 0.400*0.063*0.063
spot_data['volume_um'] = spot_data['volume']*voxel_volume_um

ax = spot_data.volume_um.plot.hist(bins=500) # rule of thumb around 1 um^3
ax.set_xlabel(r'Cell Volume [$\mu m^3$]')
ax.set_xlim([0, 8])
f = ax.get_figure()
f.savefig('CellVolume.svg')
f.savefig('CellVolume.png')


In [ ]:
spot_data.sort_values(['TRACK_ID', 'FRAME'], inplace=True)
spot_data['diffs'] = spot_data['volume_um'].diff()
spot_data

In [ ]:
mask = spot_data.TRACK_ID != spot_data.TRACK_ID.shift(1)
spot_data['diffs'][mask] = np.nan

In [ ]:
spot_data

In [ ]:
# need time deltas!
from scipy.io import loadmat
from pathlib import Path


In [ ]:
raw_data_folder = Path(r'Y:\Daniel\000_Microscope data\2020.09.15_CNN3\kdv1502R_5L_30ms_300gain002\Pos5')
raw_data_mats = sorted(raw_data_folder.glob('*ch1_frame*.mat'))
metadata = loadmat(raw_data_mats[0])

In [ ]:
times = []
for i, mat_file in enumerate(raw_data_mats):
    metadata = loadmat(mat_file)
    times.append([i, pd.to_datetime(metadata['data'][0][0][1][0])])
    
time_data = pd.DataFrame(times, columns=['FRAME', 'TIME'])

In [ ]:
time_data['time_diff'] = time_data.TIME.diff()

In [ ]:
time_data

In [ ]:
spot_data = pd.merge(spot_data, time_data,on='FRAME')

In [ ]:
spot_data['time_diff_min'] = spot_data.time_diff.astype('timedelta64[s]')/60
spot_data['time_delta_min'] = (spot_data.TIME - spot_data.TIME[0]).astype('timedelta64[s]') / 60

In [ ]:
spot_data['growthrate_um^3_per_min'] = spot_data['diffs'] /spot_data['time_diff_min']

In [ ]:
import seaborn as sns



df = spot_data

df[['mean_z', 'mean_y', 'mean_x']] = \
    df.groupby(['FRAME'])[['POSITION_Z', 'POSITION_Y', 'POSITION_X']].transform('mean')

df['distance'] = np.sum(
        (
            df[['POSITION_Z', 'POSITION_Y', 'POSITION_X']].values 
                - df[['mean_z', 'mean_y', 'mean_x']].values
        )**2,
     axis=1)

#nbins_time = len(df.FRAME.unique())
nbins_time = 50
nbins_location = 50

v = df.groupby([
    pd.cut(df["time_delta_min"], np.linspace(df['time_delta_min'].min(), df['time_delta_min'].max(), nbins_time)),
    pd.cut(df['distance'], np.linspace(df['distance'].min(), df['distance'].max(), nbins_location))
])['growthrate_um^3_per_min'].mean()

heatmap = v.unstack()

In [ ]:
ax = sns.heatmap(heatmap.transpose(), 
                 cbar_kws={'label': 'growth rate [$\mu m^3/ min^{-1}$]'},
                vmax=0.1,
                vmin=0,
                cmap="viridis_r")

ax.invert_yaxis()
ax.set_xlabel('Time [min]')
ax.set_ylabel('Distance [vox]')

xtick_labels = [np.mean([float(x) for x in text.get_text()[1:-1].split(',')]) for text in ax.get_xticklabels()]
xtick_labels = [f'{x:0.1f}' for x in xtick_labels]
ax.set_xticklabels(xtick_labels) 

ytick_labels = [np.mean([float(x) for x in text.get_text()[1:-1].split(',')]) for text in ax.get_yticklabels()]
ytick_labels = [f'{x:0.1f}' for x in ytick_labels]
ax.set_yticklabels(ytick_labels) 


f = ax.get_figure()


#output_name = Path(args.output_fig)
#output_name.parent.mkdir(parents=True, exist_ok=True)

#f.savefig(output_name)
#return

In [ ]:
import matplotlib.pyplot as plt

f, ax = plt.subplots(1, figsize=(5, 10))

for track_id in spot_data.TRACK_ID.unique()[:10]:
    sel = spot_data.TRACK_ID == track_id
    spot_data_ = spot_data[sel]
    spot_data_.sort_values('time_delta_min')
    ax.plot(spot_data_.time_delta_min, spot_data_.POSITION_Y)
    ax.set_ylabel('Position y [vox]')
    ax.set_xlabel('Time [min]')

In [ ]:
spot_data.TRACK_ID.unique()

In [ ]:
spot_data

In [ ]:
spot_data.to_csv('tracked_spots.csv')